# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234399406122                   -0.50    7.0
  2   -7.249779930668       -1.81       -1.39    1.0
  3   -7.251028858135       -2.90       -1.79    4.0
  4   -7.251017726592   +   -4.95       -1.90    2.0
  5   -7.251307919259       -3.54       -2.42    2.0
  6   -7.251337301190       -4.53       -2.97    1.0
  7   -7.251338776971       -5.83       -3.84    2.0
  8   -7.251338795033       -7.74       -4.29    2.0
  9   -7.251338798551       -8.45       -4.97    2.0
 10   -7.251338798686       -9.87       -5.42    3.0
 11   -7.251338798704      -10.74       -6.16    3.0
 12   -7.251338798705      -12.40       -6.91    2.0
 13   -7.251338798705      -14.21       -7.33    3.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05334339750138064
[ Info: Arnoldi iteration step 2: normres = 0.5091273536259958
[ Info: Arnoldi iteration step 3: normres = 0.8661256251602201
[ Info: Arnoldi iteration step 4: normres = 0.39629180636026207
[ Info: Arnoldi iteration step 5: normres = 0.5029370649071984
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (7.71e-02, 8.06e-02, 3.88e-01, 2.99e-01, 2.24e-02)
[ Info: Arnoldi iteration step 6: normres = 0.32388202330432947
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.13e-02, 1.44e-01, 2.55e-01, 9.56e-02, 9.22e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08972582395774384
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (9.50e-04, 1.54e-02, 1.67e-02, 4.23e-02, 6.60e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11340537405962953
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (4.76e